In [1]:
from utils.dataset_download import *
import pickle
import json
import requests
from pathlib import Path, PosixPath
import pandas as pd

In [2]:
# import API credentials
with open('oed_experiments/oed_credentials.json') as f:
    credentials = json.load(f)

In [3]:
lemmas = ['nation', 'art', 'technology', 'labour', 'power', 'power', 'democracy', 'woman', 'man', 'slave', 'apple', 'anger', 'happiness']

In [4]:
save_path = Path("./lemmas/")
save_path.mkdir(exist_ok=True)

In [5]:
##TO DO##
#Do the analysis using the pickles and not the request to the API. This was an initial attempt for speed..

lemma_ids = []

for lemma in lemmas:
    url = "https://oed-researcher-api.oxfordlanguages.com/oed/api/v0.2/words/?lemma=" + lemma
    response = requests.get(url, headers=credentials) 
    
    if response.status_code == 200: # check status code
        data = json.dumps(response.json()) 
        senses = json.loads(data)

        print (lemma.upper())
        print('Number of definitions', ': ', len(senses['data']))
        print()
        for definition in senses['data']:            
            print(definition['id'])
            
            lemma_id = definition['id']
            lemma_ids.append(lemma_id)

            #for each lemma ID get all senses and quotes
            sense_json = query_oed(credentials,'word',lemma_id,flags='include_senses=true&include_quotations=true')
            senses_df = convert_json_to_dataframe(sense_json)
            senses_df['n_quotes'] = senses_df.quotations.apply(lambda x: len(x))

            
            #save it
            # as pickle
            senses_df.to_pickle(save_path / f"senses_{lemma_id}.pickle")
            # as csv
            senses_df.to_csv(save_path / f"senses_{lemma_id}.tsv",sep='\t')


            #filter out the dates

            mask_start = ((senses_df['daterange.end'].isnull()) | (senses_df['daterange.end'] > 1750))
            senses_filtered_df = senses_df[mask_start]

            mask_end = ((senses_filtered_df['daterange.start'] < 1920) | senses_filtered_df['daterange.start'].isnull()) 

            senses_filtered_df = senses_filtered_df[mask_end]
            senses_filtered_df['n_quotes'] = senses_filtered_df.quotations.apply(lambda x: len(x))

            print('Number of senses' ': ', len(senses_df['id']))
            print('Number of senses (1750-1920)' ': ', len(senses_filtered_df['id']))
            print('Number of total quotes' ': ', sum(senses_df['n_quotes']))
            print('Number of total quotes (1750-1920): ' , sum(senses_filtered_df['n_quotes']))
            print()


        print()   
            
    else:
        raise Exception(f"Error while accessing the API\nResponse code={response.status_code}")
    


NATION
Number of definitions :  4

nation_nn01
Number of senses:  18
Number of senses (1750-1920):  12
Number of total quotes:  129
Number of total quotes (1750-1920):  102

nation_jj01
Number of senses:  1
Number of senses (1750-1920):  1
Number of total quotes:  6
Number of total quotes (1750-1920):  6

nation_rb01
Number of senses:  1
Number of senses (1750-1920):  1
Number of total quotes:  9
Number of total quotes (1750-1920):  9

nation_nn02
Number of senses:  2
Number of senses (1750-1920):  2
Number of total quotes:  12
Number of total quotes (1750-1920):  12


ART
Number of definitions :  4

art_nn01
Number of senses:  20
Number of senses (1750-1920):  17
Number of total quotes:  227
Number of total quotes (1750-1920):  213

art_vb01
Number of senses:  4
Number of senses (1750-1920):  0
Number of total quotes:  13
Number of total quotes (1750-1920):  0

art_nn02
Number of senses:  1
Number of senses (1750-1920):  1
Number of total quotes:  6
Number of total quotes (1750-1920):

NameError: name 'cap' is not defined